In [1]:
from quantopian.pipeline import Pipeline
from quantopian.research import run_pipeline
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.factors import SimpleMovingAverage, AverageDollarVolume

## Custom Factors
When we first looked at factors, we explored the set of built-in factors. Frequently, a desired computation isn't included as a built-in factor. One of the most powerful features of the Pipeline API is that it allows us to define our own custom factors. When a desired computation doesn't exist as a built-in, we define a custom factor.

Conceptually, a custom factor is identical to a built-in factor. It accepts `inputs`, `window_length`, and `mask` as constructor arguments, and returns a `Factor` object each day.

Let's take an example of a computation that doesn't exist as a built-in: standard deviation. To create a factor that computes the [standard deviation](https://en.wikipedia.org/wiki/Standard_deviation) over a trailing window, we can subclass `quantopian.pipeline.CustomFactor` and implement a compute method whose signature is:


```
def compute(self, today, asset_ids, out, *inputs):
    ...
```

- `*inputs` are M x N [numpy arrays](http://docs.scipy.org/doc/numpy-1.10.1/reference/arrays.ndarray.html), where M is the `window_length` and N is the number of securities (usually around ~8000 unless a `mask` is provided). `*inputs` are trailing data windows. Note that there will be one M x N array for each `BoundColumn` provided in the factor's `inputs` list. The data type of each array will be the `dtype` of the corresponding `BoundColumn`.
- `out` is an empty array of length N. `out` will be the output of our custom factor each day. The job of compute is to write output values into out.
- `asset_ids` will be an integer [array](http://docs.scipy.org/doc/numpy-1.10.0/reference/generated/numpy.array.html) of length N containing security ids corresponding to the columns in our `*inputs` arrays.
- `today` will be a [pandas Timestamp](http://pandas.pydata.org/pandas-docs/stable/timeseries.html#converting-to-timestamps) representing the day for which `compute` is being called.

Of these, `*inputs` and `out` are most commonly used.

An instance of `CustomFactor` that’s been added to a pipeline will have its compute method called every day. For example, let's define a custom factor that computes the standard deviation of the close price over the last 5 days. To start, let's add `CustomFactor` and `numpy` to our import statements.

In [2]:
from quantopian.pipeline import CustomFactor
import numpy

Next, let's define our custom factor to calculate the standard deviation over a trailing window using [numpy.nanstd](http://docs.scipy.org/doc/numpy-dev/reference/generated/numpy.nanstd.html):

In [3]:
class StdDev(CustomFactor):
    def compute(self, today, asset_ids, out, values):
        # Calculates the column-wise standard deviation, ignoring NaNs
        out[:] = numpy.nanstd(values, axis=0)

Finally, let's instantiate our factor in `make_pipeline()`:

In [4]:
def make_pipeline():
    std_dev = StdDev(inputs=[USEquityPricing.close], window_length=5)

    return Pipeline(
        columns={
            'std_dev': std_dev
        }
    )

When this pipeline is run, `StdDev.compute()` will be called every day with data as follows:

- `values`: An M x N [numpy array](http://docs.scipy.org/doc/numpy-1.10.1/reference/arrays.ndarray.html), where M is 5 (`window_length`), and N is ~8000 (the number of securities in our database on the day in question).
- `out`: An empty array of length N (~8000). In this example, the job of `compute` is to populate `out` with an array storing of 5-day close price standard deviations.

In [5]:
result = run_pipeline(make_pipeline(), '2018-05-05', '2018-05-05')
result

/venvs/py35/lib/python3.5/site-packages/numpy/lib/nanfunctions.py:1202: RuntimeWarning: Degrees of freedom <= 0 for slice.
  warnings.warn("Degrees of freedom <= 0 for slice.", RuntimeWarning)


std_dev
2018-05-07 00:00:00+00:00 Equity(2 [HWM])           0.292092
                          Equity(21 [AAME])         0.025000
                          Equity(24 [AAPL])         6.522592
                          Equity(25 [HWM_PR])       0.525000
                          Equity(31 [ABAX])         0.827406
                          Equity(41 [ARCB])         0.704344
                          Equity(52 [ABM])          0.313968
                          Equity(53 [ABMD])        19.288199
                          Equity(62 [ABT])          0.403901
                          Equity(64 [GOLD])         0.037094
                          Equity(66 [AB])           0.165367
                          Equity(67 [ADSK])         1.457990
                          Equity(70 [VBF])          0.031865
                          Equity(76 [TAP])          5.571030
                          Equity(84 [ACET])         0.096540
                          Equity(100 [IEP])         1.181118
                          Equity(106 [ACU])         0.072930
                          Equity(110 [RAMP])        0.241131
                          Equity(112 [ACY])         0.165808
                          Equity(114 [ADBE])        2.769198
                          Equity(117 [AEY])         0.006013
                          Equity(122 [ADI])         0.992636
                          Equity(128 [ADM])         0.675710
                          Equity(149 [ADX])         0.066513
                          Equity(153 [AE])          1.020127
                          Equity(154 [AEM])         0.266638
                          Equity(157 [AEG])         0.122164
                          Equity(161 [AEP])         0.286524
                          Equity(166 [AES])         0.048104
                          Equity(168 [AET])         3.114427
...                                                      ...
                          Equity(51954 [SDI])       0.000000
                          Equity(51955 [CDAY])      1.113779
                          Equity(51957 [LASR])      1.730854
                          Equity(51959 [ENBA])      0.050754
                          Equity(51960 [DOV_WI])    1.037047
                          Equity(51961 [SMAR])      0.299707
                          Equity(51962 [GSHD])      0.274000
                          Equity(51963 [ELOX])      0.455219
                          Equity(51964 [SJIU])      0.321988
                          Equity(51966 [APY])       1.020893
                          Equity(51967 [EDNT])      0.000000
                          Equity(51968 [FCBP])      0.531123
                          Equity(51969 [ECCX])      0.010000
                          Equity(51970 [BH_A])     59.511255
                          Equity(51971 [PRT])       0.313723
                          Equity(51973 [UBX])       0.073000
                          Equity(51974 [ASLN])      0.000000
                          Equity(51975 [INSP])      0.295000
                          Equity(51976 [STXB])      0.000000
                          Equity(51977 [ETP_PRC])   0.033000
                          Equity(51979 [RCP])            NaN
                          Equity(51980 [BCML])      0.000000
                          Equity(51981 [JRSH])      0.000000
                          Equity(51982 [CBLK])      0.000000
                          Equity(51983 [ROAD])      0.000000
                          Equity(51984 [SPXB])      0.000000
                          Equity(51985 [SDCI])      0.000000
                          Equity(51987 [AMRX])           NaN
                          Equity(51988 [CHEK_Z])         NaN
                          Equity(51989 [PLL])            NaN

[8568 rows x 1 columns]

### Default Inputs
When writing a custom factor, we can set default `inputs` and `window_length` in our `CustomFactor` subclass. For example, let's define the `TenDayMeanDifference` custom factor to compute the mean difference between two data columns over a trailing window using [numpy.nanmean](http://docs.scipy.org/doc/numpy-dev/reference/generated/numpy.nanmean.html). Let's set the default `inputs` to `[USEquityPricing.close, USEquityPricing.open]` and the default `window_length` to 10:

In [6]:
class TenDayMeanDifference(CustomFactor):
    # Default inputs.
    inputs = [USEquityPricing.close, USEquityPricing.open]
    window_length = 10
    def compute(self, today, asset_ids, out, close, open):
        # Calculates the column-wise mean difference, ignoring NaNs
        out[:] = numpy.nanmean(close - open, axis=0)

<i>Remember in this case that `close` and `open` are each 10 x ~8000 2D [numpy arrays.](http://docs.scipy.org/doc/numpy-1.10.1/reference/arrays.ndarray.html)</i>

If we call `TenDayMeanDifference` without providing any arguments, it will use the defaults.

In [7]:
# Computes the 10-day mean difference between the daily open and close prices.
close_open_diff = TenDayMeanDifference()

The defaults can be manually overridden by specifying arguments in the constructor call.

In [8]:
# Computes the 10-day mean difference between the daily high and low prices.
high_low_diff = TenDayMeanDifference(inputs=[USEquityPricing.high, USEquityPricing.low])

### Further Example
Let's take another example where we build a [momentum](http://www.investopedia.com/terms/m/momentum.asp) custom factor and use it to create a filter. We will then use that filter as a `screen` for our pipeline.

Let's start by defining a `Momentum` factor to be the division of the most recent close price by the close price from `n` days ago where `n` is the `window_length`.

In [9]:
class Momentum(CustomFactor):
    # Default inputs
    inputs = [USEquityPricing.close]

    # Compute momentum
    def compute(self, today, assets, out, close):
        out[:] = close[-1] / close[0]

Now, let's instantiate our `Momentum` factor (twice) to create a 10-day momentum factor and a 20-day momentum factor. Let's also create a `positive_momentum` filter returning `True` for securities with both a positive 10-day momentum and a positive 20-day momentum.

In [10]:
ten_day_momentum = Momentum(window_length=10)
twenty_day_momentum = Momentum(window_length=20)

positive_momentum = ((ten_day_momentum > 1) & (twenty_day_momentum > 1))

Next, let's add our momentum factors and our `positive_momentum` filter to `make_pipeline`. Let's also pass `positive_momentum` as a `screen` to our pipeline.

In [11]:
def make_pipeline():

    ten_day_momentum = Momentum(window_length=10)
    twenty_day_momentum = Momentum(window_length=20)

    positive_momentum = ((ten_day_momentum > 1) & (twenty_day_momentum > 1))

    std_dev = StdDev(inputs=[USEquityPricing.close], window_length=5)

    return Pipeline(
        columns={
            'std_dev': std_dev,
            'ten_day_momentum': ten_day_momentum,
            'twenty_day_momentum': twenty_day_momentum
        },
        screen=positive_momentum
    )

Running this pipeline outputs the standard deviation and each of our momentum computations for securities with positive 10-day and 20-day momentum.

In [12]:
result = run_pipeline(make_pipeline(), '2018-05-05', '2018-05-05')
result

std_dev  \
2018-05-07 00:00:00+00:00 Equity(24 [AAPL])          6.522592   
                          Equity(53 [ABMD])         19.288199   
                          Equity(64 [GOLD])          0.037094   
                          Equity(66 [AB])            0.165367   
                          Equity(67 [ADSK])          1.457990   
                          Equity(100 [IEP])          1.181118   
                          Equity(110 [RAMP])         0.241131   
                          Equity(114 [ADBE])         2.769198   
                          Equity(117 [AEY])          0.006013   
                          Equity(122 [ADI])          0.992636   
                          Equity(149 [ADX])          0.066513   
                          Equity(153 [AE])           1.020127   
                          Equity(161 [AEP])          0.286524   
                          Equity(166 [AES])          0.048104   
                          Equity(192 [ATAX])         0.022361   
                          Equity(216 [HES])          0.554530   
                          Equity(283 [ALCO])         0.229818   
                          Equity(289 [MATX])         0.808421   
                          Equity(312 [ALOT])         0.234521   
                          Equity(351 [AMD])          0.145520   
                          Equity(397 [AMSW_A])       0.180665   
                          Equity(410 [AN])           0.468231   
                          Equity(465 [APH])          0.577463   
                          Equity(526 [AROW])         0.233238   
                          Equity(542 [ASA])          0.066151   
                          Equity(559 [ASH])          2.921853   
                          Equity(600 [OA])           0.555820   
                          Equity(612 [ATO])          1.128415   
                          Equity(630 [ADP])          2.719488   
                          Equity(693 [AZO])          8.376798   
...                                                       ...   
                          Equity(51692 [SLDB])       1.609427   
                          Equity(51698 [TORC])       0.378418   
                          Equity(51700 [AGS])        0.471237   
                          Equity(51701 [UMH_PRD])    0.252290   
                          Equity(51702 [ALIN_PRE])   0.060697   
                          Equity(51709 [TMFC])       0.128533   
                          Equity(51713 [KOIN])       0.116333   
                          Equity(51736 [CMRE_PRE])   0.123191   
                          Equity(51746 [VCTR])       0.097160   
                          Equity(51777 [VFMO])       0.542513   
                          Equity(51792 [ROBT])       0.160259   
                          Equity(51802 [NEWT_I])     0.066030   
                          Equity(51813 [RFL])        0.163511   
                          Equity(51820 [BWB])        0.152918   
                          Equity(51821 [CMSA])       0.063320   
                          Equity(51827 [RCUS])       0.734330   
                          Equity(51831 [AIHS])       0.057411   
                          Equity(51832 [ZS])         1.003960   
                          Equity(51844 [DNJR])       0.235710   
                          Equity(51845 [FIXX])       0.633296   
                          Equity(51849 [IBMM])       0.051253   
                          Equity(51855 [STG])        0.295527   
                          Equity(51860 [IETC])       0.245178   
                          Equity(51866 [BDRY])       0.532180   
                          Equity(51871 [GDL_PRC])    0.114438   
                          Equity(51884 [PPTY])       0.132755   
                          Equity(51894 [GNPX])       0.659588   
                          Equity(51900 [FRLG])       1.333814   
                          Equity(51904 [CMDY])       0.116386   
                          Equity(51906 [CAI_PRA])    0.061435   

                       

Custom factors allow us to define custom computations in a pipeline. They are frequently the best way to perform computations on [partner datasets](https://www.quantopian.com/data) or on multiple data columns. The full documentation for CustomFactors is available [here](https://www.quantopian.com/help#custom-factors).

In the next lesson, we'll use everything we've learned so far to create a pipeline for an algorithm.